In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
import matplotlib.pyplot as plt
import numpy as np
import csv
from PIL import Image



device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [2]:
xs_0=list()
xs_0_=list()
xs_1=list()
ys_0=list()
data_number= 20
batch_size = 5
epochs = 2000
#100장정도.. 램 가능
with open ('data/Data_narrow.txt', 'rt' ) as r_n:
    target_r = r_n.readline()
    for j in range(data_number): 
        print(j)
        xs_0=list()
        for i in range(2): 
            temp_red=list()
            temp_blue=list()
            temp_green=list()
            if i == 0:
                continue
            r=open ('data/'+str(i+20*j)+'_building_r.csv', 'r' )
            rdr=csv.reader(r)
            for target in rdr:
                temp_red.append(target)
                
            r=open ('data/'+str(i+20*j)+'_building_g.csv', 'r' )
            rdr=csv.reader(r)
            for target in rdr:
                temp_green.append(target)
                
            r=open ('data/'+str(i+20*j)+'_building_b.csv', 'r' )
            rdr=csv.reader(r)
            for target in rdr:
                temp_blue.append(target)
                
            temp_red=np.array(temp_red, dtype=np.float32)
            temp_greed=np.array(temp_green, dtype=np.float32)
            temp_blue=np.array(temp_blue, dtype=np.float32)

            xs_1=np.stack((temp_red, temp_green, temp_blue), axis=0)
            xs_1=list(xs_1) 
            xs_0.append(xs_1)
            
            target_r = r_n.readline()
            if i<20:
                A_r,B_r,C_r,D_r,E_r,F_r,G_r,H_r,I_r,J_r,K_r=target_r.split(',')  

                if i==0:
                    pass

                else:
                    if i==1:
                        ys_0.append([float(E_r)])

                    else:

                        ys_0=ys_0[:-1]+[ys_0[-1]+[float(E_r)]]

        xs_0_.append(xs_0)

x_train = np.array(xs_0_, dtype=np.float32)
y_train = np.array(ys_0, dtype=np.float32)
x_train = torch.from_numpy(x_train).cuda()
y_train = torch.from_numpy(y_train).cuda()

xs_0=list()
xs_0_=list()
xs_1=list()
"""
for j in range(data_number): 
    xs_0=list()
    for i in range(10): 
        temp_red=list()
        temp_blue=list()
        temp_green=list()

        r=open ('data/'+str(i+10*j)+'_r.csv', 'r' )
        rdr=csv.reader(r)
        for target in rdr:
            temp_red.append(target)

        r=open ('data/'+str(i+10*j)+'_g.csv', 'r' )
        rdr=csv.reader(r)
        for target in rdr:
            temp_green.append(target)

        r=open ('data/'+str(i+10*j)+'_b.csv', 'r' )
        rdr=csv.reader(r)
        for target in rdr:
            temp_blue.append(target)

        temp_red=np.array(temp_red, dtype=np.float32)
        temp_greed=np.array(temp_green, dtype=np.float32)
        temp_blue=np.array(temp_blue, dtype=np.float32)

        xs_1=np.stack((temp_red, temp_green, temp_blue), axis=0)
        xs_1=list(xs_1) 
        xs_0.append(xs_1)
    
    xs_0_.append(xs_0)
        
x_test = np.array(xs_0_, dtype=np.float32)
x_test = torch.from_numpy(x_test).cuda()
"""
#print(x_train.shape,y_train.shape)
dataset = TensorDataset(x_train, y_train)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
#print(x_train.shape,y_train)
#dataset_test = TensorDataset(x_test, y_train)
#dataloader_test = DataLoader(dataset_test, batch_size=batch_size, shuffle=True)
#print(x_test.shape)

xs_0=list()
xs_0_=list()
xs_1=list()
ys_0=list()
temp_red=list()
temp_blue=list()
temp_green=list()

print("data_ready")

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
data_ready


In [4]:
class LRCN(nn.Module):
    def __init__(self):
        super(LRCN, self).__init__()
        self.layer_1 = nn.Conv2d(3,3,10,stride=1, padding='same')
        self.layer_2 = nn.Conv2d(3,5,10,stride=3)
        self.layer_3 = nn.Conv2d(5,2,10,stride=4)
        #self.layer_4 = nn.Conv2d(20,30,5,stride=2)
        #self.layer_5 = nn.Conv2d(30,40,5,stride=2)
        self.layer_6 = nn.LSTM(168,10,1, batch_first = True)
        self.layer_7 = nn.Linear(10, 1)
        
        self.batch_size = batch_size
        self.length = x_train.shape[1]
        self.relu = nn.ReLU()
        
        self.a=0
        self.b=0
        self.c=0
        #self.d=0
        #self.e=0
        self.count_=0
        self.o=0
        self.ct=0
        """
        for child in self.children():
            self.ct += 1
            #print(child)
            if self.ct < 3:
                for param in child.parameters():
                    param.requires_grad = False
        """
        
    def forward(self, x):
        x = x.reshape((-1, 3, 100, 200))
        x = self.layer_1(x)
        self.a = self.relu(x)
        x = self.layer_2(self.a)
        self.b = self.relu(x)
        x = self.layer_3(self.b)
        self.c = self.relu(x)
        #x = self.layer_4(self.c)
        #self.d = self.relu(x)
        #x = self.layer_5(self.d)
        #self.e = self.relu(x)
        x = self.c.reshape((self.batch_size, self.length, x.shape[-3],x.shape[-2],x.shape[-1]))
        x = x.reshape((self.batch_size, self.length, x.shape[-3]*x.shape[-2]*x.shape[-1]))
        self.o, (h_n, c_n) = self.layer_6(x)
        #print(x.shape)
        
        #x = self.relu(h_n[-1,:,:].reshape((self.batch_size,-1)))
        x = self.layer_7(self.o[:,:,:]).reshape((self.batch_size,-1))
        #print(x.shape)
        #x.retain_grad()
        self.count_+=1
        #print("종우",x.grad)
        return x

model = LRCN().cuda()
loss = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-2)
#print("서",model.parameters().shape)
      
def train(dataloader, model, loss, optimizer):
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        pred = model(X)
        
        #print(pred.shape,y.shape)
        loss_result = loss(pred, y)
        optimizer.zero_grad()
        print(pred, y,loss_result)
        model.a.retain_grad()
        model.b.retain_grad()
        model.c.retain_grad()
        #model.d.retain_grad()
        #model.e.retain_grad()
        model.o.retain_grad()
        #print(model.e.grad)
        loss_result.backward()
        
    if model.count_==4:
        temp_sum=0

        width_ = model.a.grad.shape[-1]
        result_temp = torch.sum(model.a.grad,-1)
        height_ = result_temp.shape[-1]
        result_temp = torch.sum(result_temp,-1)
        result_temp_ = torch.sum(result_temp,0)
        result_temp = result_temp_/3
        model.count_=0
    
        for k in range(1):  
            for j in range(result_temp.shape[0]):
                k=0
                temp_grad = model.a.grad.reshape((model.batch_size, model.length, 3, model.a.grad.shape[-2],model.a.grad.shape[-1]))
                temp_sum += result_temp[j]*(temp_grad[0,k,j,:,:])
            result_ = temp_sum.cpu().numpy()
            min_= np.min(result_)
            result_=result_-min_
            max_ = np.max(result_)
            min_= np.min(result_)
            result_ = ((255*result_)/max_)#.astype(np.int16)
            min_= np.min(result_)
            max_ = np.max(result_)
            pil_image=Image.fromarray(result_)
            pil_image.show()
            with open("data/"+str(i)+"_r.csv",'a',newline='') as aa:
                writer_a = csv.writer(aa, delimiter=',')
            
            result_ = model.relu(temp_sum.cpu()).numpy()
            min_= np.min(result_)
            result_=result_-min_
            max_ = np.max(result_)
            min_= np.min(result_)
            result_ = ((255*result_)/max_)#.astype(np.int16)
            min_= np.min(result_)
            max_ = np.max(result_)
            pil_image=Image.fromarray(result_)
            pil_image.show()
            
            
            
    optimizer.step()
            
def test(dataloader, model, loss):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss(pred, y).item()
    test_loss /= num_batches
    print(f"Test Error: \n Avg loss: {test_loss:>8f} \n")

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(dataloader, model, loss, optimizer)
    #test(dataloader_test, model, loss)
print("Done!")


Epoch 1
-------------------------------
tensor([[0.2346],
        [0.2447],
        [0.1833],
        [0.2407],
        [0.2467]], device='cuda:0', grad_fn=<ReshapeAliasBackward0>) tensor([[4.6100],
        [5.2376],
        [5.2595],
        [2.3507],
        [2.6225]], device='cuda:0') tensor(15.9874, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor([[0.2424],
        [0.2322],
        [0.2370],
        [0.2354],
        [0.2396]], device='cuda:0', grad_fn=<ReshapeAliasBackward0>) tensor([[2.7675],
        [2.6009],
        [5.2334],
        [3.9330],
        [0.0063]], device='cuda:0') tensor(10.1356, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor([[0.2386],
        [0.2292],
        [0.1757],
        [0.2384],
        [0.2183]], device='cuda:0', grad_fn=<ReshapeAliasBackward0>) tensor([[2.6186],
        [1.7734],
        [0.0612],
        [0.0000],
        [2.6170]], device='cuda:0') tensor(2.7747, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor([[0.1897],
        [0.24

<ipython-input-4-5bfbcb4f3654>:102: RuntimeWarning: invalid value encountered in true_divide
  result_ = ((255*result_)/max_)#.astype(np.int16)
<ipython-input-4-5bfbcb4f3654>:115: RuntimeWarning: invalid value encountered in true_divide
  result_ = ((255*result_)/max_)#.astype(np.int16)


Epoch 3
-------------------------------
tensor([[0.2096],
        [0.2096],
        [0.2096],
        [0.2096],
        [0.2096]], device='cuda:0', grad_fn=<ReshapeAliasBackward0>) tensor([[2.6170],
        [2.6186],
        [0.0612],
        [2.3507],
        [0.4338]], device='cuda:0') tensor(3.2511, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor([[0.2096],
        [0.2096],
        [0.2096],
        [0.2096],
        [0.2096]], device='cuda:0', grad_fn=<ReshapeAliasBackward0>) tensor([[1.0767],
        [2.6009],
        [2.6225],
        [0.0000],
        [4.6100]], device='cuda:0') tensor(6.3398, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor([[0.2096],
        [0.2096],
        [0.2096],
        [0.2096],
        [0.2096]], device='cuda:0', grad_fn=<ReshapeAliasBackward0>) tensor([[5.0909],
        [0.0063],
        [3.2547],
        [3.9330],
        [5.2376]], device='cuda:0') tensor(14.4572, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor([[0.2096],
        [0.209

KeyboardInterrupt: 